# 주유소 10곳이상 기름값 지도위에 표시하기

**오피넷 - 경로별**   
- 출발지: 서울, 연신내역
- 도착지: 서울, 호서직업전문학교 - 주유소 3개
- 도착지: 서울, 서울역 - 주유소 1개
- 도착지: 서울, 김포국제공항 - 주유소 4개
- 도착지: 서울, 여의도역 - 주유소 5개

In [1]:
import numpy as np
import pandas as pd
import time

# Unix style pathname pattern expansion
from glob import glob

# A browser automation framework and ecosystem.
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller 

# Beautiful Soup is a library that makes it easy to scrape information from web pages.
from bs4 import BeautifulSoup

# A Fast, Extensible Progress Bar for Python and CLI
from tqdm import tqdm
from tqdm import tqdm_notebook

## Crawling

In [2]:
path = chromedriver_autoinstaller.install() 
driver = webdriver.Chrome(path)
driver.switch_to.window(driver.window_handles[0])

# 메인페이지 접속
driver.get("http://www.opinet.co.kr/user/main/mainView.do")

# 싼주유소찾기 이동
wait = WebDriverWait(driver, 10)
quick_ul = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#quick_ul')))
quick_ul.find_element_by_css_selector('li:nth-child(2) a').click()

# 경로별 페이지 이동
wait = WebDriverWait(driver, 10)
header_path_gsi = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#container .header_path_gsi')))
header_path_gsi.find_element_by_css_selector('li.mid2 a').click()

# 출발지, 도착지 설정
destinations = ['호서직업전문학교', '서울역', '김포국제공항', '여의도역']
departures = ['연신내'] * len(destinations)

for i, destination in tqdm(enumerate(destinations)):

    # 경로별 페이지 설정
    wait = WebDriverWait(driver, 10)
    form = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#searrouteVO')))

    # 경로별 설정 - 출발지
    departure = departures[i]

    element = form.find_element_by_id('search1_2')
    driver.execute_script("arguments[0].style.display = 'none';", element)

    time.sleep(1)
    Select(form.find_element_by_id('SIDO_CD1')).select_by_visible_text('서울')
    form.find_element_by_id('poiName1').send_keys('')
    form.find_element_by_id('poiName1').send_keys(departure)
    form.find_element_by_id('poiName1').send_keys(Keys.ENTER)

    # 경로별 설정 - 도착지
    element = form.find_element_by_id('search2_2')
    driver.execute_script("arguments[0].style.display = 'none';", element)

    time.sleep(1)
    Select(form.find_element_by_id('SIDO_CD2')).select_by_visible_text('서울')
    form.find_element_by_id('poiName2').send_keys('')
    form.find_element_by_id('poiName2').send_keys(destination)
    form.find_element_by_id('poiName2').send_keys(Keys.ENTER)

    # 경로별 설정 - 조회
    time.sleep(3)
    driver.find_element_by_id('searrouteSelect').click()

    # 경로별 설정 - 엑셀 저장
    time.sleep(5)
    driver.find_element_by_id('excelIcon').click()

    # to refresh the browser
    time.sleep(3)
    driver.refresh()

driver.close() 
driver.quit()

4it [00:56, 14.02s/it]


## Move the download file to a working directory.

In [3]:
import shutil

move_files = 'C:\\Users\\shoseo\\Downloads\\*.xls'
move_path = 'D:\\Workspace\\Data-Analysis-2020\\05.CrawlingOpinet\\data'

shutil.rmtree(move_path)
os.mkdir('./data')

for move_from in glob(move_files):
    filename = move_from.split('\\')[-1]
    move_to = move_path +'\\'+ filename
    shutil.move(move_from, move_to)

## Merging DataFrame

In [14]:
pd.read_html('./data/경로별 주유소현황.xls')[0]

,순번,주유소명,고급휘발유,휘발유,경유
0,1,다회주유소,-,1304,1104
1,2,지에스칼텍스㈜ 신사제일주유소,1485,1315,1115
2,3,대성석유(주) 마곡대성주유소,1599,1365,1185


In [12]:
tmp_raw = []

for filepath in glob('./data/*.xls'):
    tmp = pd.read_html(filepath)[0]
    tmp_raw.append(tmp)

tmp_df = pd.concat(tmp_raw)
tmp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 2
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   순번      13 non-null     int64 
 1   주유소명    13 non-null     object
 2   고급휘발유   13 non-null     object
 3   휘발유     13 non-null     int64 
 4   경유      13 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 624.0+ bytes


In [15]:
df = pd.DataFrame({
    '주유소명': tmp_df['주유소명'],
    '고급휘발유': tmp_df['고급휘발유'],
    '휘발유': tmp_df['휘발유'],
    '경유': tmp_df['경유']
})
df.head()

,주유소명,고급휘발유,휘발유,경유
0,(주)창원이엔지 홍제지점,-,1299,1099
0,행촌에너지 공항주유소,-,1264,1064
1,(주)유카스 에너지 올림픽점,-,1265,1065
2,산성주유소,-,1275,1075
3,(주)한진 김포공항주유소,-,1319,1128
